# Import libraries

In [3]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
#!pip install openpyxl

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re

from lhcsmapi.Time import Time
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery
from scipy import stats

from src.acquisitions.pc_pm import PCPM
from src.acquisitions.voltage_nqps import VoltageNQPS

from src.utils.utils import log_acquisition
from src.utils.hdf_tools import acquisition_to_hdf5, load_from_hdf_with_regex
from src.visualisation.visualisation import plot_hdf

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

# Select events to download

In [3]:
# Define list of relevant tests (from https://twiki.cern.ch/twiki/bin/viewauth/MP3/FPAinRB)
list_timestamp_fgc_string = {
    # Cases :  2kA@10A/s,             2kA flat top,            5kA@10A/s,           5kA flat top,          8kA@10A/s,             8kA flat top,          11kA@10A/s,            11kA flat top
    'RB.A12': ['20210929-171841.860', '20220324-142234.020', '20220317-173754.160', '20220318-180302.900', '20220324-113250.100', '20220317-082527.640', '20220321-174349.880', '20220314-182111.500',], # RB.A12
    'RB.A23': ['20211007-172946.940', '20220513-151726.480', '20220420-154328.720', '20220513-164703.480', '20220516-172626.320', '20220311-164815.980', '20220420-094801.980', '20220412-110941.500',], # RB.A23
    'RB.A34': ['20210917-165956.200', '20220321-173018.160', '20220317-173859.020', '20220317-143906.740', '20220317-114529.560', '20220317-082141.440', '20220315-180726.000', '20220314-180355.300',], # RB.A34
    'RB.A45': ['20210714-172843.720', '20220324-172052.620', '20220317-173955.340', '20220324-161221.920', '20220324-142807.040', '20220317-082212.300', '20220315-180946.660', '20220314-180432.060',], # RB.A45
    'RB.A56': ['20210715-172329.660', '20220325-133158.060', '20220317-174047.920', '20220324-171334.780', '20220324-152927.080', '20220317-082233.180', '20220315-180837.380', '20220314-180456.120',], # RB.A56
    'RB.A67': ['20210716-165040.520', '20220310-104434.420', '20220310-112544.240', '20220303-175132.120', '20220310-143922.380', '20220310-120722.100', '20220311-170356.700', '20220314-180525.040',], # RB.A67
    'RB.A78': ['20220225-164757.500', '20220310-104452.800', '20220310-125030.720', '20220310-112559.500', '20220310-165252.980', '20220310-143749.600', '20220321-172944.960', '20220311-170212.220',], # RB.A78
    'RB.A81': ['20210811-171210.020', '20220324-142253.660', '20220324-171155.840', '20220325-121835.180', '20220317-173556.780', '20220324-152810.860', '20220321-172902.140', '20220324-113328.260',], # RB.A81
    }
list_cases = ['2kA@10A/s', '2kA flat top', '5kA@10A/s', '5kA flat top', '8kA@10A/s', '8kA flat top', '11kA@10A/s', '11kA flat top']  # list of relevant cases

In [4]:
fpa_identifiers = ['RB_RB.A12_1632928721860000000',
                 'RB_RB.A12_1647278471500000000',
                 'RB_RB.A12_1647501927640000000',
                 'RB_RB.A12_1647535074160000000',
                 'RB_RB.A12_1647622982900000000',
                 'RB_RB.A12_1647881029880000000',
                 'RB_RB.A12_1648117970100000000',
                 'RB_RB.A12_1648128154020000000',
                 'RB_RB.A23_1633620586940000000',
                 'RB_RB.A23_1647013695980000000',
                 'RB_RB.A23_1649754581500000000',
                 'RB_RB.A23_1650440881980000000',
                 'RB_RB.A23_1650462208720000000',
                 'RB_RB.A23_1652447846480000000',
                 'RB_RB.A23_1652453223480000000',
                 'RB_RB.A23_1652714786320000000',
                 'RB_RB.A34_1631890796200000000',
                 'RB_RB.A34_1647277435300000000',
                 'RB_RB.A34_1647364046000000000',
                 'RB_RB.A34_1647501701440000000',
                 'RB_RB.A34_1647513929560000000',
                 'RB_RB.A34_1647524346740000000',
                 'RB_RB.A34_1647535139020000000',
                 'RB_RB.A34_1647880218160000000',
                 'RB_RB.A45_1626276523720000000',
                 'RB_RB.A45_1647277472060000000',
                 'RB_RB.A45_1647364186660000000',
                 'RB_RB.A45_1647501732300000000',
                 'RB_RB.A45_1647535195340000000',
                 'RB_RB.A45_1648128487040000000',
                 'RB_RB.A45_1648134741920000000',
                 'RB_RB.A45_1648138852620000000',
                 'RB_RB.A56_1626362609660000000',
                 'RB_RB.A56_1647277496120000000',
                 'RB_RB.A56_1647364117380000000',
                 'RB_RB.A56_1647501753180000000',
                 'RB_RB.A56_1647535247920000000',
                 'RB_RB.A56_1648132167080000000',
                 'RB_RB.A56_1648138414780000000',
                 'RB_RB.A56_1648211518060000000',
                 'RB_RB.A67_1626447040520000000',
                 'RB_RB.A67_1646326292120000000',
                 'RB_RB.A67_1646905474420000000',
                 'RB_RB.A67_1646907944240000000',
                 'RB_RB.A67_1646910442100000000',
                 'RB_RB.A67_1646919562380000000',
                 'RB_RB.A67_1647014636700000000',
                 'RB_RB.A67_1647277525040000000',
                 'RB_RB.A78_1645804077500000000',
                 'RB_RB.A78_1646905492800000000',
                 'RB_RB.A78_1646907959500000000',
                 'RB_RB.A78_1646913030720000000',
                 'RB_RB.A78_1646919469600000000',
                 'RB_RB.A78_1646927572980000000',
                 'RB_RB.A78_1647014532220000000',
                 'RB_RB.A78_1647880184960000000',
                 'RB_RB.A81_1628694730020000000',
                 'RB_RB.A81_1647534956780000000',
                 'RB_RB.A81_1647880142140000000',
                 'RB_RB.A81_1648118008260000000',
                 'RB_RB.A81_1648128173660000000',
                 'RB_RB.A81_1648132090860000000',
                 'RB_RB.A81_1648138315840000000',
                 'RB_RB.A81_1648207115180000000']

In [5]:
signal_groups = [PCPM, VoltageNQPS]
file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals')

## Test query

In [ ]:
# quench with precursor
fpa_identifier = {'circuit_type': 'RB',
                 'circuit_name': 'RB.A81',
                 'timestamp_fgc': 1648207115180000000}

In [ ]:
context_path = file_dir / "20221123_context_snapshots"

for signal_group in signal_groups:
    group = signal_group(**fpa_identifier, spark=spark)
    acquisition_to_hdf5(acquisition=group, 
                        file_dir=file_dir, 
                        context_dir_name="20221123_snapshots_context",
                        failed_queries_dir_name="20221123_snapshots_failed",
                        data_dir_name="")
        
log_acquisition(identifier=fpa_identifier, log_data={"download_complete": True}, log_path=context_path)

## Load data

In [ ]:
file_name = f"RB_RB.A81_1648207115180000000.hdf5"
file_path = file_dir / Path("20221123_snapshots_data") / file_name

signals = ['I_MEAS','VoltageNQPS.*U_DIODE']
len(signals)

In [ ]:
data = load_from_hdf_with_regex(file_path)

In [ ]:
plot_hdf(data, column_regex=signals)

## Query all FGC events

In [6]:
signals = ['I_MEAS','VoltageNQPS.*U_DIODE']

In [13]:
for f in fpa_identifiers:
    
    fpa_identifier = {'circuit_type': 'RB',
                     'circuit_name': f.split("_")[1],
                     'timestamp_fgc': int(f.split("_")[2])}
    
    file_name = f"{f}.hdf5"
    Path(file_dir / Path('20221123_snapshots_plots')).mkdir(parents=True, exist_ok=True)
    
    if not os.path.isfile(str(file_dir / Path('20221123_snapshots_data') / f"{f}.png")):
        for signal_group in signal_groups:
            group = signal_group(**fpa_identifier)
            acquisition_to_hdf5(acquisition=group, 
                                file_dir=file_dir,
                                context_dir_name="20221123_snapshots_context",
                                failed_queries_dir_name="20221123_snapshots_failed",
                                data_dir_name="20221123_snapshots_data")

        file_path = file_dir / Path('20221123_snapshots_data') / file_name
        data = load_from_hdf_with_regex(file_path)
        fig = plt.figure(figsize=(20, 15))
        for i, s in enumerate(signals):
            fig.add_subplot(4, 3, i+1)
            n_signals = 0
            for df in data:
                if bool(re.search(s, df.columns.values[0])):
                    plt.plot(df.index.values, df.values)
                    n_signals += 1
            plt.title(f"{s} ({n_signals})")

        plt.savefig(str(file_dir / Path('20221123_snapshots_data') / f"{f}.png"))
        plt.close()
    else:
        print(f"done: {file_name}")

	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA23.RB.A12 at 2021-09-29 17:18:41.860
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2021-09-29 17:18:31.860 to 2021-09-29 17:25:21.860


Post Mortem returned no data for the following query: http://pm-rest.cern.ch/v2//pmdata/header/within/duration?system=QPS&className=DQAMGNSRB_PMREL&source=*&fromTimestampInNanos=1632928711860000000&durationInNanos=410000000000
Some nQPS PM buffers missing. ['B8R1', 'B9R1', 'B10R1', 'B11R1', 'B12R1', 'B13R1', 'B14R1', 'B15R1', 'B16R1', 'B17R1', 'B18R1', 'B19R1', 'B20R1', 'B21R1', 'B22R1', 'B23R1', 'B24R1', 'B25R1', 'B26R1', 'B27R1', 'B28R1', 'B29R1', 'B30R1', 'B31R1', 'B32R1', 'B33R1', 'B34R1', 'B34L2', 'B33L2', 'B32L2', 'B31L2', 'B30L2', 'B29L2', 'B28L2', 'B27L2', 'B26L2', 'B25L2', 'B24L2', 'B23L2', 'B22L2', 'B21L2', 'B20L2', 'B19L2', 'B18L2', 'B17L2', 'B16L2', 'B15L2', 'B14L2', 'B13L2', 'B12L2', 'B11L2', 'B10L2', 'B9L2', 'B8L2'] not found for RB.A12 within [(10, 's'), (400, 's')] around 1632928721860000000.


	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA23.RB.A12 at 2022-03-14 18:21:11.500
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-14 18:21:01.500 to 2022-03-14 18:27:51.500


Some nQPS PM buffers missing. ['B13R1'] not found for RB.A12 within [(10, 's'), (400, 's')] around 1647278471500000000.


	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA23.RB.A12 at 2022-03-17 08:25:27.640
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-17 08:25:17.640 to 2022-03-17 08:32:07.640


...

	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA83.RB.A78 at 2022-03-10 11:25:59.500
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-10 11:25:49.500 to 2022-03-10 11:32:39.500
	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA83.RB.A78 at 2022-03-10 12:50:30.720
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-10 12:50:20.720 to 2022-03-10 12:57:10.720
	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA83.RB.A78 at 2022-03-10 14:37:49.600
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-10 14:37:39.600 to 2022-03-10

Post Mortem returned no data for the following query: http://pm-rest.cern.ch/v2//pmdata/header/within/duration?system=QPS&className=DQAMGNSRB_PMREL&source=*&fromTimestampInNanos=1628694720020000000&durationInNanos=410000000000
Some nQPS PM buffers missing. ['B8R8', 'B9R8', 'B10R8', 'B11R8', 'B12R8', 'B13R8', 'B14R8', 'B15R8', 'B16R8', 'B17R8', 'B18R8', 'B19R8', 'B20R8', 'B21R8', 'B22R8', 'B23R8', 'B24R8', 'B25R8', 'B26R8', 'B27R8', 'B28R8', 'B29R8', 'B30R8', 'B31R8', 'B32R8', 'B33R8', 'B34R8', 'B34L1', 'B33L1', 'B32L1', 'B31L1', 'B30L1', 'B29L1', 'B28L1', 'B27L1', 'B26L1', 'B25L1', 'B24L1', 'B23L1', 'B22L1', 'B21L1', 'B20L1', 'B19L1', 'B18L1', 'B17L1', 'B16L1', 'B15L1', 'B14L1', 'B13L1', 'B12L1', 'B11L1', 'B10L1', 'B9L1', 'B8L1'] not found for RB.A81 within [(10, 's'), (400, 's')] around 1628694730020000000.


	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA87.RB.A81 at 2022-03-17 17:35:56.780
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-17 17:35:46.780 to 2022-03-17 17:42:36.780
	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA87.RB.A81 at 2022-03-21 17:29:02.140
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-21 17:28:52.140 to 2022-03-21 17:35:42.140
	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA87.RB.A81 at 2022-03-24 11:33:28.260
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-24 11:33:18.260 to 2022-03-24

Some nQPS PM buffers missing. ['B8R8', 'B9R8', 'B10R8', 'B11R8', 'B12R8', 'B13R8', 'B14R8', 'B15R8', 'B16R8', 'B17R8', 'B18R8', 'B19R8', 'B20R8', 'B21R8', 'B22R8', 'B23R8', 'B24R8', 'B25R8', 'B26R8', 'B27R8', 'B28R8', 'B29R8', 'B30R8', 'B31R8', 'B32R8', 'B33R8', 'B34R8', 'B34L1', 'B33L1', 'B32L1', 'B31L1', 'B30L1', 'B29L1', 'B28L1', 'B27L1', 'B26L1', 'B25L1', 'B24L1', 'B23L1', 'B22L1', 'B21L1', 'B20L1', 'B19L1', 'B18L1', 'B17L1', 'B16L1', 'B15L1', 'B14L1', 'B13L1', 'B12L1', 'B11L1', 'B10L1', 'B9L1', 'B8L1'] not found for RB.A81 within [(10, 's'), (400, 's')] around 1648128173660000000.


	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA87.RB.A81 at 2022-03-24 15:28:10.860
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-24 15:28:00.860 to 2022-03-24 15:34:50.860
	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA87.RB.A81 at 2022-03-24 17:11:55.840
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-24 17:11:45.840 to 2022-03-24 17:18:35.840
	Querying PM event signal(s) STATUS.I_MEAS, IAB.I_A, IEARTH.I_EARTH, STATUS.I_EARTH_PCNT, STATUS.I_REF for system: FGC, className: lhc_self_pmd, source: RPTE.UA87.RB.A81 at 2022-03-25 12:18:35.180
	Querying PM event timestamps for system: QPS, className: DQAMGNSRB_PMREL, source: * from 2022-03-25 12:18:25.180 to 2022-03-25

# Write Metadata

In [18]:
df = pd.DataFrame({"fpa_identifier":fpa_identifiers})
df["Circuit"] = df["fpa_identifier"].apply(lambda x: x.split("_")[1])
df["timestamp_fgc"] = df["fpa_identifier"].apply(lambda x: x.split("_")[2])

In [18]:
#df.to_excel("snapshots.xlsx", index=False)
df = pd.read_excel("snapshots.xlsx")

In [20]:
signals = ['I_MEAS']
for fpa_identifier in fpa_identifiers:
    data_dir = file_dir / Path('20221123_snapshots_data') / (fpa_identifier + ".hdf5")
    data_pm = load_from_hdf_with_regex(file_path=data_dir, regex_list=signals)
    
    bool_idx = df.fpa_identifier == fpa_identifier
    df.loc[bool_idx, "I_Q_M"] = data_pm[0].max().values[0]
    df.loc[bool_idx, "dt_I_Q_M"] = data_pm[0].idxmax().values[0]
    
    k, intercept, r, p, std_err = stats.linregress(data_pm[0].loc[-1:0].loc[-1:0].index.values, data_pm[0].loc[-1:0].loc[-1:0].values.reshape(-1))
    df.loc[bool_idx, "dI_dt_from_data"] = k

# Add features with NXCALS

In [47]:
k=0
for fpa_identifier in fpa_identifiers:
    
    circuit_name= fpa_identifier.split("_")[1]
    timestamp_fgc= int(fpa_identifier.split("_")[2])
    
    
    loc_mask = df.fpa_identifier == fpa_identifier

    rb_query = RbCircuitQuery("RB", circuit_name, max_executions=45)
    
    source_timestamp_ee_even_df = rb_query.find_source_timestamp_ee(timestamp_fgc, system='EE_EVEN')
    timestamp_ee_even = source_timestamp_ee_even_df.timestamp[0]
    u_dump_res_even_df = rb_query.query_ee_u_dump_res_pm(timestamp_ee_even, timestamp_fgc, system='EE_EVEN', signal_names=['U_DUMP_RES'])

    source_timestamp_ee_odd_df = rb_query.find_source_timestamp_ee(timestamp_fgc, system='EE_ODD')
    timestamp_ee_odd = source_timestamp_ee_odd_df.timestamp[0]
    u_dump_res_odd_df = rb_query.query_ee_u_dump_res_pm(timestamp_ee_odd, timestamp_fgc, system='EE_ODD', signal_names=['U_DUMP_RES'])

    timestamp_pic = rb_query.find_timestamp_pic(timestamp_fgc, spark=spark)

    source_timestamp_qds_df = rb_query.find_source_timestamp_qds(timestamp_fgc, duration=[(50, 's'), (500, 's')])
    timestamp_iQps = source_timestamp_qds_df.timestamp[0]

    # sometimes values are missing in the end
    #ts_iQPS_PIC = 1e-6 * (source_timestamp_qds_df['timestamp'].values - min(timestamp_pic))
    #df.loc[loc_mask, 'Delta_t(iQPS-PIC)'] = [ts_iQPS_PIC] #np.append(ts_iQPS_PIC, np.max(ts_iQPS_PIC)*np.ones(sum(loc_mask)-len(ts_iQPS_PIC)))

    df.loc[loc_mask, 'Delta_t(EE_odd-PIC)'] = (timestamp_ee_odd - min(timestamp_pic)) / 1e6
    df.loc[loc_mask, 'Delta_t(EE_even-PIC)'] = (timestamp_ee_even - min(timestamp_pic)) / 1e6
    df.loc[loc_mask, 'U_EE_max_ODD'] = u_dump_res_odd_df.max()[0]
    df.loc[loc_mask, 'U_EE_max_EVEN'] = u_dump_res_even_df.max()[0]
    #except Exception as e:
    #    print(e)
    #    k+=1
print(k)

	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: UA23.RB.A12 from 2021-09-29 17:18:31.860 to 2021-09-29 17:35:21.860


	Querying PM event signal(s) UA23.RB.A12:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: UA23.RB.A12 at 2021-09-29 17:18:42.426


	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR17.RB.A12 from 2021-09-29 17:18:31.860 to 2021-09-29 17:35:21.860


	Querying PM event signal(s) RR17.RB.A12:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: RR17.RB.A12 at 2021-09-29 17:18:41.930


	Querying NXCALS signal(s) RB.A12.ODD:ST_ABORT_PIC, RB.A12.EVEN:ST_ABORT_PIC from 2021-09-29 17:18:40.860 to 2021-09-29 17:19:41.860


...


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR13.RB.A81 from 2022-03-25 12:18:25.180 to 2022-03-25 12:35:15.180


	Querying PM event signal(s) RR13.RB.A81:U_DUMP_RES for system: QPS, className: DQAMSNRB, source: RR13.RB.A81 at 2022-03-25 12:18:35.258


	Querying NXCALS signal(s) RB.A81.ODD:ST_ABORT_PIC, RB.A81.EVEN:ST_ABORT_PIC from 2022-03-25 12:18:34.180 to 2022-03-25 12:19:35.180


	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2022-03-25 12:17:45.180 to 2022-03-25 12:26:55.180
0


# Add other features

In [5]:
df = pd.read_csv("../data/RB_snapshot_context.csv")

In [12]:
df["date"] = df["timestamp_fgc"].apply(lambda x: Time.to_string(x))

In [13]:
df.to_csv("../data/RB_snapshot_context.csv")